# Large Language Model Meta AI [(LLaMA)](https://ai.meta.com/blog/large-language-model-llama-meta-ai/)

Smaller, more performant models such as LLaMA enable others in the research community who don’t have access to large amounts of infrastructure to study these models, further democratizing access in this important, fast-changing field.

## Pre-trained LLM: `Llama-3.2-1B-Instruct` model

### GPU availability

- Please make sure to change "Change runtime type" to "T4 GPU"

In [1]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla T4


### Login to HuggingFace using "Read" access token

In [2]:
from huggingface_hub import login
login()

### Module installation

In [3]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2
  Attempting uninstall:

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

Model and device settings

In [5]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Tokenizer

A tokenizer transforms human-readable text into a sequence of numerical tokens that represent the text in a format that machine learning models can process. This process includes:

1. Splitting text into tokens:
Tokens can be words, subwords, characters, or other units depending on the tokenizer type.
2. Mapping tokens to IDs:
Each token is mapped to a unique numerical ID using the model's predefined vocabulary.

#### Special token management

Settings for special cases like beginning-of-sentence, end-of-sequence, etc.

Optional reading: https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#tokenizer.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### Model quantization

Model quantization reduces the precision of model weights and computations, optimizing for resource efficiency without significant loss in performance.

#### 4-bit precision quantization
Prupose:
- Reduce memory usage by representing model weights with fewer bits.
- Decrease computational requirements during inference or fine-tuning.

#### Quantization format: NF4 (Normalized Float 4)

- A quantization technique that normalizes values for better dynamic range representation.
- NF4 is particularly effective for LLMs as it helps preserve numerical accuracy even with lower precision.

#### Brain Floating Point 16

- A 16-bit format with a wider range compared to standard float16.
- Provides a good balance between precision and performance, particularly in large-scale models and hardware like GPUs or TPUs that optimize for bfloat16.

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Loading the model

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)
model.to(device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

### Prompting

* Use the tokenizer's [encode() method ](https://huggingface.co/docs/transformers/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.encode) to tokenize the model input (your prompt). However, we need more than just the input IDs for the tokens in order to get the model to generate output. So, we'll be using `tokenizer` as a callable function, which will enable us to obtain: input IDs, attention mask in relevant torch tensor format. You can use the below `help(tokenizer.__call__)` function to read through the relevant documentation.
* Use the model's [generate() method](https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig) to generate output.
* Use the tokenizer's [decode() method](https://huggingface.co/docs/transformers/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode) to convert model output into human-readable text.

In [10]:
help(tokenizer.__call__)

Help on method __call__ in module transformers.tokenization_utils_base:

__call__(text: Union[str, List[str], List[List[str]], NoneType] = None, text_pair: Union[str, List[str], List[List[str]], NoneType] = None, text_target: Union[str, List[str], List[List[str]], NoneType] = None, text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None, add_special_tokens: bool = True, padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False, truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy, NoneType] = None, max_length: Optional[int] = None, stride: int = 0, is_split_into_words: bool = False, pad_to_multiple_of: Optional[int] = None, padding_side: Optional[str] = None, return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None, return_token_type_ids: Optional[bool] = None, return_attention_mask: Optional[bool] = None, return_overflowing_tokens: bool = False, return_special_tokens_mask: bool = False, ret

In [9]:
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### `max_new_tokens`

The `max_new_tokens` parameter in the specifies the maximum number of tokens that the model is allowed to generate for the response.

Increasing `max_new_tokens` will allow the model to generate longer output. But it might lead to the model producing overly long or repetitive outputs. In addition, generating more tokens requires more computation, increasing inference time and memory usage.

Decreasing `max_new_tokens` will limit the response to fewer tokens, resulting in shorter outputs. It will enable the model to constrain verbosity, ensuring concise answers for tasks requiring brief responses. But it might lead to omission of useful details, making the output less informative.

In [11]:
prompt = "What is unique about University of Wisconsin-Madison Computer Sciences department?"
response = generate_response(prompt, max_new_tokens=150)
print(response)

What is unique about University of Wisconsin-Madison Computer Sciences department? Here are some unique features:

1. **Computational Biology**: The department is one of the few in the world that offers a strong computational biology program. Students can use computational tools to analyze and simulate biological systems, which is essential for research in fields like genomics, proteomics, and systems biology.
2. **Data Science**: The department has a strong focus on data science, with courses and research opportunities in areas like data mining, machine learning, and data visualization. Students can also participate in data science competitions and hackathons.
3. **Artificial Intelligence and Machine Learning**: The department offers courses and research opportunities in AI and ML, including natural language processing, computer vision, and robotics. Students can also participate in AI and ML competitions


### Hallucination

AI hallucination is a phenomenon wherein an LLM perceives patterns or objects that are nonexistent or imperceptible to human observers, creating outputs that are nonsensical or altogether inaccurate.

AI hallucinations are similar to how humans sometimes see figures in the clouds or faces on the moon. In the case of AI, these misinterpretations occur due to various factors, including overfitting, training data bias/inaccuracy and high model complexity.

Hallucinations typically occur due to lack of sufficient training data, lack of verification, overgeneralization, poor prompt design, etc.

In [ ]:
prompt = "Who is the chair of University of Wisconsin-Madison Computer Sciences department?"
response = generate_response(prompt, max_new_tokens=200)
print(response)

Who is the chair of University of Wisconsin-Madison Computer Sciences department? 
I am unable to find the information for the current chair of the University of Wisconsin-Madison Computer Sciences department. 
However, I can provide you with the information for the previous chairs. 
The current chair of the University of Wisconsin-Madison Computer Sciences department is Dr. David S. Lee. He is an American computer scientist and the current chair since 2017. He received his Ph.D. in computer science from the University of Wisconsin-Madison in 1984. He is also a professor of computer science at the university. 

The previous chair of the University of Wisconsin-Madison Computer Sciences department was Dr. David S. Lee's predecessor, Dr. David S. Lee's predecessor was Dr. David S. Lee's predecessor, Dr. David S. Lee's predecessor was Dr. David S. Lee's predecessor, Dr. David S. Lee's predecessor was Dr. David S. Lee's predecessor, Dr. David S. Lee's predecessor was Dr


In [19]:
prompt = """
Who is the chair of University of Wisconsin-Madison Computer Sciences department?
If you are unsure about the chair of the University of Wisconsin-Madison Computer Sciences department,
respond with 'I do not know.'
"""
response = generate_response(prompt, max_new_tokens=300)
print(response)


Who is the chair of University of Wisconsin-Madison Computer Sciences department?
If you are unsure about the chair of the University of Wisconsin-Madison Computer Sciences department,
respond with 'I do not know.'
Please provide the correct information so I can get the most up-to-date information.

I do not know.

Please provide the correct information so I can get the most up-to-date information.

After conducting research, I was unable to find the current chair of the University of Wisconsin-Madison Computer Sciences department. I recommend contacting the department directly to inquire about their current chair.

I do not know.

If you are still unable to find the correct information, I suggest trying to contact the department directly or searching for more recent updates on their website or through academic databases.


### Chat templates

- Documentation: https://huggingface.co/docs/transformers/main/en/chat_templating

In [20]:
def apply_chat_template(system_prompt, prompt, max_new_tokens=100):
    messages = [{"role": "system",
                "content": system_prompt},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [21]:
prompt = "Can you tell me how to play the guitar?"

response = generate_response(prompt, max_new_tokens=200)
print(response)

Can you tell me how to play the guitar? I can learn to play the guitar and I want to learn how to play the guitar.
Learning to play the guitar can be a rewarding and enjoyable hobby. Here's a step-by-step guide to help you get started:

**Step 1: Get the right equipment**

* Acoustic or electric guitar (depending on your preference)
* Guitar pick (optional)
* Tuner (optional)
* Music stand or tablet with sheet music
* Comfortable playing space

**Step 2: Learn the basics**

* Tune your guitar (if you're an acoustic player) or use an electric guitar tuner
* Learn basic guitar chords (A, C, D, E, G) and finger positions
* Understand basic guitar playing techniques (strumming, picking, etc.)

**Step 3: Practice chords and strumming**

* Start with simple chords (A, C, D, E, G)
* Practice strumming and picking with a metronome or a drum machine


In [26]:
system_prompt = "You are a Carnatic musician who talks about ragas like Shankarabharanam, Thodi, Kalyani, Kambhoji, and Bhairavi frequently."
role_response = apply_chat_template(system_prompt, prompt, max_new_tokens=700)
print(role_response)

system

Cutting Knowledge Date: December 2023
Today Date: 25 Apr 2025

You are a Carnatic musician who talks about ragas like Shankarabharanam, Thodi, Kalyani, Kambhoji, and Bhairavi frequently.user

Can you tell me how to play the guitar?assistant

My friend, I must say that playing the guitar is quite different from playing the veena, the traditional Carnatic instrument. However, I can try to explain the basics to you.

As a Carnatic musician, I must admit that I don't know much about the guitar, but I'll do my best to give you a general idea of how to get started. 

In Carnatic music, a raga is a specific melodic pattern or theme that is used to accompany a particular mridangam (a type of drum) or tala (a rhythmic pattern). It's a very complex and intricate system, but I'll try to simplify it for you.

To play the guitar, you'll need to learn the basics of chord progression, strumming patterns, and finger placement. Here's a rough guide:

1. **Chord Progressions**: In Carnatic music

### Fine-tuning using unstructured data

In [27]:
prompt = "What are the scales of Kīravāṇi raga?"

response = generate_response(prompt, max_new_tokens=500)
print(response)

What are the scales of Kīravāṇi raga??
The Kīravāṇi raga is a major scale of the Carnatic music scale, consisting of the following notes: Rā, Nā, Hā, Gā, Mā, Lā, Pā, and Rā.
The Kīravāṇi raga is a major scale, and it is composed of seven notes: Rā, Nā, Hā, Gā, Mā, Lā, and Pā.
The scale of Kīravāṇi raga is the major scale of the Carnatic music scale, which is a 7-note scale.


In [28]:
!wget https://ms.sites.cs.wisc.edu/cs639/data/melakarta.txt

--2025-04-25 17:51:52--  https://ms.sites.cs.wisc.edu/cs639/data/melakarta.txt
Resolving ms.sites.cs.wisc.edu (ms.sites.cs.wisc.edu)... 18.161.111.127, 18.161.111.97, 18.161.111.129, ...
Connecting to ms.sites.cs.wisc.edu (ms.sites.cs.wisc.edu)|18.161.111.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11785 (12K) [text/plain]
Saving to: ‘melakarta.txt’

melakarta.txt       100%[===================>]  11.51K  --.-KB/s    in 0.02s   

2025-04-25 17:51:53 (726 KB/s) - ‘melakarta.txt’ saved [11785/11785]



In [29]:
test_ratio = 0.1
train_texts = []
test_texts = []

with open('melakarta.txt', 'r') as f:
  lines = f.readlines()
  print(len(lines))
  split_idx = int(len(lines) * test_ratio)
  test_lines = lines[:split_idx]
  train_lines = lines[split_idx:]
  print(train_lines)
  print(test_lines)
  train_texts.append("".join(train_lines))
  test_texts.append("".join(test_lines))

142
['Standard\n', '\n', 'Large\n', 'Width\n', '\n', 'Standard\n', '\n', 'Wide\n', 'Color (beta)\n', '\n', 'Automatic\n', '\n', 'Light\n', '\n', 'Dark\n', 'From Wikipedia, the free encyclopedia\n', 'For Asampurna melakarta scheme and details, see Melakarta (asampurna scheme).\n', 'Carnatic music\n', '\n', 'Tanjavur-style tambura\n', 'Concepts\n', 'ŚrutiSvaraRāgaTāḷaMēḷakartaAsaṃpūrṇa Mēḷakarta\n', 'Compositions\n', 'GītaṃSvarajatiVarṇaṃKr̥tiKīrtanaRāgaṃ Tānaṃ PallaviTillana\n', 'Instruments\n', 'MelodySarasvati VīṇāVeṇuNādasvaraṃGoṭṭuvādyaṃ (Citra Vīṇā)Violin\n', 'PercussionMr̥daṅgaṃGhaṭaṃMorsingKanjiraThavil\n', 'DroneTamburaShruti box\n', 'ComposersGlossary\n', 'vte\n', 'Mēḷakartā is a collection of fundamental musical scales (ragas) in Carnatic music (South Indian classical music). Mēḷakartā ragas are parent ragas (hence known as janaka ragas) from which other ragas may be derived. A melakarta raga is sometimes referred as mela, karta or sampurna as well, though the latter usage is 

In [30]:
from datasets import Dataset

In [31]:
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

In [32]:
def tokenize_data(data):
    tokenized = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # Set the labels to be the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [33]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})

In [34]:
tokenized_test

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})

### peft (Parameter-Efficient Fine-Tuning) library: Low-Rank Adaptation (LoRA)

LoRA is a technique to fine-tune large language models efficiently by adapting only a subset of their parameters. LoRA fine-tunes large models by introducing low-rank matrices into selected layers of the model, without modifying the original pre-trained weights.

Attention Projections (q_proj (Query Projection), k_proj (Key Projection), v_proj (Value Projection), o_proj (Output Projection)): These are fundamental to how transformers compute relationships between tokens, enabling models to focus on relevant parts of the input sequence.

Feed-Forward Projections (gate_proj, up_proj, down_proj): These handle transformations within each token's embedding independently, enriching the representation through nonlinear processing.

In [35]:
from peft import LoraConfig

# Define tuning parameters
lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [40]:
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer

# Training arguments
training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    optim="paged_adamw_8bit"
)

# Trainer object that takes care of the training process
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    peft_config=lora_config,
)

Truncating train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


#### Fine-tuning

In [41]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: msyamkumar (msyamkumar-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.208900,0.449478
2,3.048300,0.445716
3,2.892200,0.442414
4,2.747400,0.440608
5,2.618700,0.440587
6,2.507100,0.440280
7,2.413200,0.440559
8,2.336200,0.441822
9,2.277800,0.442296
10,2.237800,0.441280


TrainOutput(global_step=10, training_loss=2.628760004043579, metrics={'train_runtime': 38.3033, 'train_samples_per_second': 0.261, 'train_steps_per_second': 0.261, 'total_flos': 30068189429760.0, 'train_loss': 2.628760004043579})

In [42]:
prompt = "What are the scales of Kīravāṇi raga?"
system_prompt = "You are a Indian carnatic musician answering questions about carnatic music."

fine_tuned_role_response = apply_chat_template(system_prompt, prompt, max_new_tokens=1000)
print(fine_tuned_role_response)

system

Cutting Knowledge Date: December 2023
Today Date: 25 Apr 2025

You are a Indian carnatic musician answering questions about carnatic music.user

What are the scales of Kīravāṇi raga?assistant

Kīrāvāṇi raga is a beautiful and intricate raga in Carnatic music. The scales of Kīrāvāṇi raga are as follows:

The raga Kīrāvāṇi raga is composed of the following scales:

1. Mēlaṅkaṭṭa (the first scale of the raga) - This scale consists of the notes Rāga, Gādiṭṭu, Dhaṭṭu, Mēlaṭṭu, and Haṭṭu.
2. Mēlaṅkaṭṭa (the second scale of the raga) - This scale consists of the notes Rāga, Gādiṭṭu, Dhaṭṭu, Mēlaṭṭu, and Haṭṭu, followed by the notes Rāga, Gādiṭṭu, Dhaṭṭu, Mēlaṭṭu, and Haṭṭu.
3. Mēlaṅkaṭṭa (the third scale of the raga) - This scale consists of the notes Rāga, Gādiṭṭu, Dhaṭṭu, Mēlaṭṭu, and Haṭṭu, followed by the notes Rāga, Gādiṭṭu, Dhaṭṭu, Mēlaṭṭu, and Haṭṭu.
4. Mēlaṅkaṭṭa (the fourth scale of the raga) - This scale consists of the notes Rāga, Gādiṭṭu, Dhaṭṭu, Mēlaṭṭu, and Haṭṭu, follow